In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# import streamlit as st

In [ ]:
df_precos = pd.read_excel('Case Dados - Analise de Ações.xlsx', sheet_name='PREÇOS')
df_setor = pd.read_excel('Case Dados - Analise de Ações.xlsx', sheet_name='SETOR')
df_macro = pd.read_excel('Case Dados - Analise de Ações.xlsx', sheet_name='MACRO')

In [ ]:
df_setor.head(10)


In [ ]:
df_precos.head(10)

In [ ]:
df_macro.head(10)

In [ ]:
df_precos.info()
df_setor.info()
df_macro.info()

In [ ]:
df_macro[df_macro.isna().any(axis=1)]
df_precos[df_precos.isna().any(axis=1)]
df_setor[df_setor.isna().any(axis=1)]
df_precos[(df_precos['NOME'].isna() | df_precos['PAPEL'].isna() | df_precos['PREÇO (R$)'].isna() | df_precos['DATA'].isna())]

In [ ]:
df_macro.isna().sum()

In [ ]:
df_macro.isna().sum() / len(df_macro)

# Transformando/ juntando os dados

In [ ]:
df_precos_setor = pd.merge(df_precos, df_setor, how='outer', on='PAPEL')
df_precos_setor_macro12 = pd.merge(df_precos_setor, df_macro, how='right', on='DATA').fillna('macro')
df_precos_setor_macro6 = pd.merge(df_precos_setor, df_macro, how='left', on='DATA').fillna('macro')

# Criando colunas com os retornos

In [ ]:

df_precos_setor_macro12['RETORNOS_SIMPLES'] = df_precos_setor_macro12.groupby(df_precos_setor_macro12.index)['PREÇO (R$)'].pct_change()
df_precos_setor_macro12['RETORNOS_ACUMULADOS'] = df_precos_setor_macro12.groupby(df_precos_setor_macro12.index)['RETORNOS_SIMPLES'].transform(lambda x : ((x + 1).cumprod() - 1))


In [ ]:
df_precos_setor_macro6.set_index('PAPEL', drop=True, inplace=True)

In [ ]:
df_precos_setor_macro6['RETORNOS_SIMPLES'] = df_precos_setor_macro6.groupby(df_precos_setor_macro6.index)['PREÇO (R$)'].pct_change()
df_precos_setor_macro6['RETORNOS_LOGARITIMOS'] = np.log(df_precos_setor_macro6.groupby(df_precos_setor_macro6.index)['PREÇO (R$)'].pct_change() + 1)

df_precos_setor_macro6['RETORNOS_ACUMULADOS'] = df_precos_setor_macro6.groupby(df_precos_setor_macro6.index)['RETORNOS_SIMPLES'].transform(lambda x : ((x + 1).cumprod() - 1))
df_precos_setor_macro6.head(15)


# pivotando a tabela simples para plotar fgrafico de performance

In [ ]:
df_precos_pivotada = df_precos.pivot_table(index='DATA', columns='PAPEL', values='PREÇO (R$)')
df_precos_pivotada

# normalizando os preços

In [ ]:
df_precos_pivotada_normalizada = df_precos_pivotada / df_precos_pivotada.iloc[0]
df_precos_pivotada_normalizada

#### ----

### com o PX

#### ----

# TOP Maiores retornos / menores retornos

In [ ]:
df_precos_setor_macro6.head(10)

In [ ]:
primeira_cotacao = df_precos_setor_macro6.groupby(df_precos_setor_macro6.index)['PREÇO (R$)'].first()
primeira_cotacao.head(6)

In [ ]:
ultima_cotacao = df_precos_setor_macro6.groupby(df_precos_setor_macro6.index)['PREÇO (R$)'].last()
ultima_cotacao.head(6)

In [ ]:
top_valorizadas = ((ultima_cotacao - primeira_cotacao) / primeira_cotacao).sort_values(ascending=False)
# df_precos_setor_macro6[['RETORNOS_ACUMULADOS']].groupby(df_precos_setor_macro6.index).last()

In [ ]:
top_desvalorizadas = ((ultima_cotacao - primeira_cotacao) / primeira_cotacao).sort_values(ascending=True)

### Calculando Rentabilidade BRUTA em R$

In [ ]:
ganho_reais = ultima_cotacao - primeira_cotacao
ganho_reais

# Calculando o risco / volatilidade da ação no periodo analisado

### calculo do desvio padrao mensal

In [ ]:
volatilidade = df_precos_setor_macro6['RETORNOS_SIMPLES'].dropna().groupby(df_precos_setor_macro6.dropna().index).std(ddof=1) #ddof=1 amostral e populacional = 0
volatilidade
# maiores_volatilidades = volatilidade

In [ ]:
df_precos_pivotada['IPCA'] = df_precos_setor_macro6['IPCA'][0:6].values
df_precos_pivotada['SELIC'] = df_precos_setor_macro6['SELIC'][0:6].values
df_precos_pivotada['SELIC_acumulada'] = df_precos_setor_macro6['SELIC'][0:6].values.cumsum()
df_precos_pivotada['IPCA_acumulado'] = df_precos_setor_macro6['IPCA'][0:6].values.cumsum()

df_precos_pivotada['USD'] = df_precos_setor_macro6['USD'][0:6].values

df_precos_pivotada

In [ ]:
df_precos_setor_macro6['IPCA'][0:6].values

# Calculando o indice de sharpe
- Supondo que retorno livre de risco +- 4,5%

In [ ]:
risco_free = 0.045
sharpe = (top_valorizadas.sort_index() - risco_free) / volatilidade.sort_index()
sharpe.head(40)

# Max Drowndown calculando:

In [ ]:
precos_max_dd = df_precos_setor_macro6.groupby(df_precos_setor_macro6.index)['PREÇO (R$)'].cummax()
# precos = df_precos_setor_macro6.groupby(df_precos_setor_macro6)[['PREÇO (R$)']]

precos_max_dd.head(6)

In [ ]:
precos = df_precos_setor_macro6['PREÇO (R$)']
precos.head(6)

In [ ]:
drowdowns = ((precos_max_dd - precos) / precos_max_dd)
drowdowns.head(60)
drowdowns_neg = ((precos - precos_max_dd ) / precos_max_dd)
drowdowns_neg.head(60)

### Maiore DD's

In [ ]:
max_dd = drowdowns_neg.groupby(drowdowns.index).min()
max_dd.sort_values().head(40)


In [ ]:
lista_piores_dd = list(max_dd.sort_values().index)
lista_piores_dd

## calculando os valores brutos

In [ ]:
drowdowns_bruto = precos_max_dd - precos
drowdowns_neg_bruto = precos - precos_max_dd

max_dd_bruto = drowdowns_neg_bruto.groupby(drowdowns_bruto.index).min()
max_dd_bruto.sort_values().head(40)

In [ ]:
lista_piores_dd_bruto = list(max_dd_bruto.sort_values().index)
lista_piores_dd_bruto

In [ ]:
df_media = df_precos_setor_macro6.groupby(df_precos_setor_macro6.index)['PREÇO (R$)'].mean().sort_values(ascending=False)
df_media.head(20)

# Juntando todos dados no DF

In [ ]:
df_completo = df_precos_setor_macro6.copy(deep=True)
df_completo

#### JUNTANDO SHARPE

In [ ]:
t1 = pd.merge(df_completo, sharpe.to_frame(name='SHARPE'), right_index=True,left_index=True, how='outer')
t1

#### JUNTANDO MEDIA

In [ ]:
t2 = pd.merge(t1, df_media.to_frame(name='MÉDIA'), right_index=True,left_index=True, how='outer')
t2

#### JUNTANDO MAX DROW DOWN

In [ ]:
t3 = pd.merge(t2,max_dd.to_frame(name='MÁXIMO DROW DOWN'), right_index=True,left_index=True, how='outer')
t3

#### JUNTANDO MAXIMO DROW DOWN BRUTO

In [ ]:
t4 = pd.merge(t3,max_dd_bruto.to_frame(name='MÁXIMO DROW DOWN bruto'), right_index=True,left_index=True, how='outer')
t4

#### JUNTANDO VOLATILIDADE

In [ ]:
t5 = pd.merge(t4,volatilidade.to_frame(name='VOLATILIDADE'), right_index=True,left_index=True, how='outer')
t5

#### JUNTANDO RETORNO DO PERIODO

In [ ]:
t6 = pd.merge(t5,top_valorizadas.to_frame(name='RENTABILIDADE'), right_index=True,left_index=True, how='outer')
t6

#### JUNTANDO RENTABILIDADE BRUTA DO PERIODO

In [ ]:
t7 = pd.merge(t6,ganho_reais.to_frame(name='GANHO BRUTO R$'), right_index=True,left_index=True, how='outer')
t7

In [211]:
df = t7.copy(deep=True)
df

,DATA,NOME,PREÇO (R$),SETOR ECONÔMICO,SUBSETOR,SEGMENTO,IPCA,SELIC,USD,RETORNOS_SIMPLES,RETORNOS_LOGARITIMOS,RETORNOS_ACUMULADOS,SHARPE,MÉDIA,MÁXIMO DROW DOWN,MÁXIMO DROW DOWN bruto,VOLATILIDADE,RENTABILIDADE,GANHO BRUTO R$
PAPEL,,,,,,,,,,,,,,,,,,,
ABEV3,2023-01-31,AMBEV S/A,13.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0053,0.0112,5.0987,NaN,NaN,NaN,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
ABEV3,2023-02-28,AMBEV S/A,13.39,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0084,0.0092,5.2072,-0.019766,-0.019964,-0.019766,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
ABEV3,2023-03-31,AMBEV S/A,14.33,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0071,0.0117,5.0798,0.070202,0.067847,0.049048,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
ABEV3,2023-04-30,AMBEV S/A,14.16,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0061,0.0092,5.0001,-0.011863,-0.011934,0.036603,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
ABEV3,2023-05-31,AMBEV S/A,14.41,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0023,0.0112,5.0953,0.017655,0.017501,0.054905,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YDUQ3,2023-02-28,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0084,0.0092,5.2072,-0.317522,-0.382025,-0.317522,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49
YDUQ3,2023-03-31,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0071,0.0117,5.0798,0.000000,0.000000,-0.317522,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49
YDUQ3,2023-04-30,YDUQS PART,8.12,Consumo Cíclico,Diversos,Serviços Educacionais,0.0061,0.0092,5.0001,0.151773,0.141303,-0.213940,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49


# Começando as analises e plot de graficos

In [ ]:
from plotly import graph_objects as go
from plotly.subplots import make_subplots


In [ ]:
df.info()

# PLOT TODAS AÇÕES PARA VIZUALIZAÇÃO BASICA

In [ ]:
import plotly.express as px

fig = px.line(df_precos_pivotada, x=df_precos_pivotada.index, y=df_precos_pivotada.columns, title='Evolução do Preço das Ações ao Longo do Tempo', markers='marker')#height=6000)

fig.update_layout(
    title='Evolução do Preço das Ações ao Longo do Tempo SEM NORMALIZAR',
    legend_title_text='TICKER',
    xaxis_title='Data',
    yaxis_title='Preço'
)

fig.update_xaxes(title_text='Data') # ALTERNATIVA
fig.update_yaxes(title_text='Preço') # ALTERNATIVA

fig.show()

# Plot de grafico acoes com retornos positivos NO FINAL DO PERIODO DE 6 MESES

In [ ]:
ret_maiores_que_1 = df_precos_pivotada_normalizada.loc[:, df_precos_pivotada_normalizada.loc[df_precos_pivotada_normalizada.index[-1]] >= 1]
ret_maiores_que_1

figura = go.Figure()

for col in ret_maiores_que_1.columns:
    trace = go.Scatter(x=ret_maiores_que_1.index, y=ret_maiores_que_1[col], mode='lines+markers', name=col)
    figura.add_trace(trace)

figura.update_layout(title='Evolução do Preço das Ações ao Longo do Tempo')
figura.update_layout(legend_title_text='TICKER')

figura.update_layout(
    title='Evolução do Preço das Ações ao Longo do Tempo',
    legend_title_text='TICKER',
    xaxis_title='Data',
    yaxis_title='Preço',
    height=1000
)

figura.show()

# Plot de grafico acoes com retornos negativos NO FINAL DO PERIODO DE 6 MESES

In [ ]:
ret_menores_que_1 = df_precos_pivotada_normalizada.loc[:, df_precos_pivotada_normalizada.loc[df_precos_pivotada_normalizada.index[-1]] <= 1]
ret_menores_que_1

figura = go.Figure()

for col in ret_menores_que_1.columns:
    trace = go.Scatter(x=ret_menores_que_1.index, y=ret_menores_que_1[col], mode='lines+markers', name=col)
    figura.add_trace(trace)

figura.update_layout(title='Evolução do Preço das Ações ao Longo do Tempo COM RENTABILIDADE RUIM')
figura.update_layout(legend_title_text='TICKER')

figura.update_layout(
    title='Evolução do Preço das Ações ao Longo do Tempo',
    legend_title_text='TICKER',
    xaxis_title='Data',
    yaxis_title='Preço',
    height=1000
)

figura.show()

# ANÁLISESSSSSS @@@@@@@@@@@@@@@@@

In [ ]:
import streamlit as st 

In [ ]:
df.to_excel('df_completo.xlsx')

In [ ]:
df.head()

## Por SETOR ECONOMICO

In [ ]:
df.groupby('SETOR ECONÔMICO')['PREÇO (R$)'].mean().sort_values(ascending=False)

In [ ]:
df.groupby('SETOR ECONÔMICO')['SHARPE'].mean().sort_values(ascending=False)


In [ ]:
df.groupby('SETOR ECONÔMICO')['VOLATILIDADE'].mean().sort_values(ascending=True)


In [ ]:
df.groupby('SETOR ECONÔMICO')['VOLATILIDADE'].mean().sort_values(ascending=True)


In [ ]:
df[df.index == 'POMO4']

In [ ]:
df.groupby(['SETOR ECONÔMICO', 'SUBSETOR'])['SHARPE'].mean()

In [ ]:
df['SEGMENTO'].value_counts()


In [ ]:
df['SEGMENTO'].value_counts(normalize=True)

## Por SUB SETOR 

In [ ]:
df.groupby('SUBSETOR')['PREÇO (R$)'].mean().sort_values(ascending=False)

# TESTES GRAFICO DE BARRAS

In [66]:
tickers = ['ABEV3', 'YDUQ3','AZUL4']
best_tickers = df.loc[: , ['MÉDIA' ,'SHARPE', 'MÁXIMO DROW DOWN', 'MÁXIMO DROW DOWN bruto', 'VOLATILIDADE', 'RENTABILIDADE', 'GANHO BRUTO R$']].idxmax().values

max_values = df.loc[: , ['MÉDIA' ,'SHARPE', 'MÁXIMO DROW DOWN', 'MÁXIMO DROW DOWN bruto', 'VOLATILIDADE', 'RENTABILIDADE', 'GANHO BRUTO R$']].max().values
info = df.loc[: , ['MÉDIA' ,'SHARPE', 'MÁXIMO DROW DOWN', 'MÁXIMO DROW DOWN bruto', 'VOLATILIDADE', 'RENTABILIDADE', 'GANHO BRUTO R$']].max().index.values


In [67]:
best_tickers, info, max_values

(array(['VALE3', 'POMO4', 'POMO4', 'POMO4', 'AZUL4', 'YDUQ3', 'AZUL4'],
       dtype=object),
 array(['MÉDIA', 'SHARPE', 'MÁXIMO DROW DOWN', 'MÁXIMO DROW DOWN bruto',
        'VOLATILIDADE', 'RENTABILIDADE', 'GANHO BRUTO R$'], dtype=object),
 array([76.75833333,  6.3193426 ,  0.        ,  0.        ,  0.45179676,
         0.91868345, 10.01      ]))

In [68]:
tudu_juntu = zip(best_tickers, info, max_values)
lista_tudu_junto = list(tudu_juntu)

In [69]:
ltj = lista_tudu_junto
ltj

[('VALE3', 'MÉDIA', 76.75833333333334),
 ('POMO4', 'SHARPE', 6.3193426033136655),
 ('POMO4', 'MÁXIMO DROW DOWN', 0.0),
 ('POMO4', 'MÁXIMO DROW DOWN bruto', 0.0),
 ('AZUL4', 'VOLATILIDADE', 0.45179675954297854),
 ('YDUQ3', 'RENTABILIDADE', 0.9186834462729913),
 ('AZUL4', 'GANHO BRUTO R$', 10.01)]

In [158]:
lista_acoes = ['AESB3', 'ABEV3']

In [171]:
df.loc[df.index == 'ABEV3', info]

,MÉDIA,SHARPE,MÁXIMO DROW DOWN,MÁXIMO DROW DOWN bruto,VOLATILIDADE,RENTABILIDADE,GANHO BRUTO R$
PAPEL,,,,,,,
ABEV3,14.226667,1.92827,-0.019766,-0.27,0.043101,0.128111,1.75
ABEV3,14.226667,1.92827,-0.019766,-0.27,0.043101,0.128111,1.75
ABEV3,14.226667,1.92827,-0.019766,-0.27,0.043101,0.128111,1.75
ABEV3,14.226667,1.92827,-0.019766,-0.27,0.043101,0.128111,1.75
ABEV3,14.226667,1.92827,-0.019766,-0.27,0.043101,0.128111,1.75
ABEV3,14.226667,1.92827,-0.019766,-0.27,0.043101,0.128111,1.75


In [180]:
figura = go.Figure()

# Adiciona barras para cada indicador
for acao in lista_acoes:
    figura.add_trace(go.Bar(name=acao, x=info, 
                            y=round(df.loc[df.index == acao, info],2),
                            marker_color='blue',)
                            )
    
    
# Layout do gráfico
figura.update_layout(barmode='group', title='Valores Máximos por Indicador e Ação',
                     xaxis_title='Informações', yaxis_title='Valor',)
figura.show()

In [152]:
info = ['MÉDIA', 'SHARPE', 'MÁXIMO DROW DOWN', 'MÁXIMO DROW DOWN bruto', 'VOLATILIDADE', 'RENTABILIDADE', 'GANHO BRUTO R$']
best_tickers = ['VALE3', 'POMO4', 'POMO4', 'POMO4', 'AZUL4', 'YDUQ3', 'AZUL4']
max_values = [76.76, 6.32, 0.0, 0.0, 0.45, 0.92, 10.01]

# Criação do gráfico de barras empilhadas
figura = go.Figure()

# Adiciona barras para cada indicador
for i, informacao in enumerate(info):
    # Use best_tickers[i] como o valor para o eixo x
    text_values = [f"{informacao}: {value}" for value in max_values]
    figura.add_trace(go.Bar(name=best_tickers[i], x=[informacao], y=[max_values[i]], marker_color='blue'))

# Layout do gráfico
# figura.update_traces(hoverinfo='text+y+x')
figura.update_layout(barmode='group', title='Valores Máximos por Indicador e Ação',
                     xaxis_title='Indicadores', yaxis_title='Valor')

# Exibe o gráfico
figura.show()

In [184]:
tickers = ['ABEV3', 'YDUQ3','AZUL4']
best_tickers = df.loc[: , ['MÉDIA' ,'SHARPE', 'MÁXIMO DROW DOWN', 'MÁXIMO DROW DOWN bruto', 'VOLATILIDADE', 'RENTABILIDADE', 'GANHO BRUTO R$']].idxmax().values

max_values = df.loc[: , ['MÉDIA' ,'SHARPE', 'MÁXIMO DROW DOWN', 'MÁXIMO DROW DOWN bruto', 'VOLATILIDADE', 'RENTABILIDADE', 'GANHO BRUTO R$']].max().values
info = df.loc[: , ['MÉDIA' ,'SHARPE', 'MÁXIMO DROW DOWN', 'MÁXIMO DROW DOWN bruto', 'VOLATILIDADE', 'RENTABILIDADE', 'GANHO BRUTO R$']].max().index.values

acoes_selecionadas = tickers
acoes_selecionadas
info
figura = go.Figure()

# Adiciona barras para cada indicador
for acao in acoes_selecionadas:
    figura.add_trace(go.Bar(name=acao, x=info, 
                            y=round(df.loc[df.index == acao, info],2),
                            marker_color='blue',)
                            )
    
    
# Layout do gráfico
figura.update_layout(barmode='group', title='Valores Máximos por Indicador e Ação',
                    xaxis_title='Informações', yaxis_title='Valor',)
# st.plotly_chart(figura)
figura.show()

In [ ]:
todos_tickers = df.index
todos_tickers = todos_tickers.value_counts().index
todos_tickers

In [210]:
df_completo

,DATA,NOME,PREÇO (R$),SETOR ECONÔMICO,SUBSETOR,SEGMENTO,IPCA,SELIC,USD,RETORNOS_SIMPLES,RETORNOS_LOGARITIMOS,RETORNOS_ACUMULADOS
PAPEL,,,,,,,,,,,,
ABEV3,2023-01-31,AMBEV S/A,13.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0053,0.0112,5.0987,NaN,NaN,NaN
ABEV3,2023-02-28,AMBEV S/A,13.39,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0084,0.0092,5.2072,-0.019766,-0.019964,-0.019766
ABEV3,2023-03-31,AMBEV S/A,14.33,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0071,0.0117,5.0798,0.070202,0.067847,0.049048
ABEV3,2023-04-30,AMBEV S/A,14.16,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0061,0.0092,5.0001,-0.011863,-0.011934,0.036603
ABEV3,2023-05-31,AMBEV S/A,14.41,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0023,0.0112,5.0953,0.017655,0.017501,0.054905
...,...,...,...,...,...,...,...,...,...,...,...,...
YDUQ3,2023-02-28,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0084,0.0092,5.2072,-0.317522,-0.382025,-0.317522
YDUQ3,2023-03-31,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0071,0.0117,5.0798,0.000000,0.000000,-0.317522
YDUQ3,2023-04-30,YDUQS PART,8.12,Consumo Cíclico,Diversos,Serviços Educacionais,0.0061,0.0092,5.0001,0.151773,0.141303,-0.213940


In [223]:
import plotly.graph_objects as go
from plotly import data

df_teste = data.medals_wide()

fig = go.Figure()

# Iterar sobre as colunas do DataFrame
for col in ['gold', 'silver', 'bronze']:
    fig.add_trace(go.Bar(x=df_teste.nation, y=df_teste[col], name=col))

fig.update_layout(barcornerradius=15)
fig.show()

In [285]:
df = t7.copy(deep=True)
df.reset_index(inplace=True)
df

,PAPEL,DATA,NOME,PREÇO (R$),SETOR ECONÔMICO,SUBSETOR,SEGMENTO,IPCA,SELIC,USD,RETORNOS_SIMPLES,RETORNOS_LOGARITIMOS,RETORNOS_ACUMULADOS,SHARPE,MÉDIA,MÁXIMO DROW DOWN,MÁXIMO DROW DOWN bruto,VOLATILIDADE,RENTABILIDADE,GANHO BRUTO R$
0,ABEV3,2023-01-31,AMBEV S/A,13.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0053,0.0112,5.0987,NaN,NaN,NaN,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
1,ABEV3,2023-02-28,AMBEV S/A,13.39,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0084,0.0092,5.2072,-0.019766,-0.019964,-0.019766,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
2,ABEV3,2023-03-31,AMBEV S/A,14.33,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0071,0.0117,5.0798,0.070202,0.067847,0.049048,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
3,ABEV3,2023-04-30,AMBEV S/A,14.16,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0061,0.0092,5.0001,-0.011863,-0.011934,0.036603,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
4,ABEV3,2023-05-31,AMBEV S/A,14.41,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0023,0.0112,5.0953,0.017655,0.017501,0.054905,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,YDUQ3,2023-02-28,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0084,0.0092,5.2072,-0.317522,-0.382025,-0.317522,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49
524,YDUQ3,2023-03-31,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0071,0.0117,5.0798,0.000000,0.000000,-0.317522,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49
525,YDUQ3,2023-04-30,YDUQS PART,8.12,Consumo Cíclico,Diversos,Serviços Educacionais,0.0061,0.0092,5.0001,0.151773,0.141303,-0.213940,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49
526,YDUQ3,2023-05-31,YDUQS PART,14.07,Consumo Cíclico,Diversos,Serviços Educacionais,0.0023,0.0112,5.0953,0.732759,0.549715,0.362052,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49


In [264]:
import time

In [322]:
info

array(['MÉDIA', 'SHARPE', 'MÁXIMO DROW DOWN', 'MÁXIMO DROW DOWN bruto',
       'VOLATILIDADE', 'RENTABILIDADE', 'GANHO BRUTO R$'], dtype=object)

In [344]:
df

,PAPEL,DATA,NOME,PREÇO (R$),SETOR ECONÔMICO,SUBSETOR,SEGMENTO,IPCA,SELIC,USD,RETORNOS_SIMPLES,RETORNOS_LOGARITIMOS,RETORNOS_ACUMULADOS,SHARPE,MÉDIA,MÁXIMO DROW DOWN,MÁXIMO DROW DOWN bruto,VOLATILIDADE,RENTABILIDADE,GANHO BRUTO R$
0,ABEV3,2023-01-31,AMBEV S/A,13.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0053,0.0112,5.0987,NaN,NaN,NaN,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
1,ABEV3,2023-02-28,AMBEV S/A,13.39,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0084,0.0092,5.2072,-0.019766,-0.019964,-0.019766,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
2,ABEV3,2023-03-31,AMBEV S/A,14.33,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0071,0.0117,5.0798,0.070202,0.067847,0.049048,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
3,ABEV3,2023-04-30,AMBEV S/A,14.16,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0061,0.0092,5.0001,-0.011863,-0.011934,0.036603,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
4,ABEV3,2023-05-31,AMBEV S/A,14.41,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0023,0.0112,5.0953,0.017655,0.017501,0.054905,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,YDUQ3,2023-02-28,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0084,0.0092,5.2072,-0.317522,-0.382025,-0.317522,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49
524,YDUQ3,2023-03-31,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0071,0.0117,5.0798,0.000000,0.000000,-0.317522,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49
525,YDUQ3,2023-04-30,YDUQS PART,8.12,Consumo Cíclico,Diversos,Serviços Educacionais,0.0061,0.0092,5.0001,0.151773,0.141303,-0.213940,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49
526,YDUQ3,2023-05-31,YDUQS PART,14.07,Consumo Cíclico,Diversos,Serviços Educacionais,0.0023,0.0112,5.0953,0.732759,0.549715,0.362052,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49


In [342]:
fig = go.Figure()

# Iterar sobre as colunas do DataFrame
for col in ['gold', 'silver', 'bronze']:
    fig.add_trace(go.Bar(x=df_teste.nation, y=df_teste[col], name=col))

fig.update_layout(barcornerradius=15)
fig.show()

In [288]:
df_teste = data.medals_wide()

fig = make_subplots(rows=1, cols=len(acoes_selecionadas), shared_yaxes=True)


# Iterar sobre as colunas do DataFrame
for i, col in enumerate(df_teste.columns[1:], start=1):
    fig.add_trace(
        go.Bar(x=df_teste.nation, y=df_teste[col], name=col, ),
        row=1, col=i
    )

fig.show()